In [7]:
import pandas as pd

from config import DATA_FILE_PATH, CHUNKSIZE


data_chunks = pd.read_csv(DATA_FILE_PATH, chunksize= CHUNKSIZE)

In [21]:
# Load in chunks
num_chunks = 0

# Process chunks
for idx, chunk in enumerate(data_chunks): 
    num_chunks += 1

print(num_chunks)
print(idx)
print(chunk['canonical_smiles'])

0
74
37888    CC(=O)O[C@@]12CO[C@@H]1C[C@H](O)[C@@]1(C)C(=O)...
37889    CO[C@@]12[C@H](COC(N)=O)C3=C(C(=O)C(C)=C(N)C3=...
37890    CC[C@@]1(O)C(=O)OCc2c1cc1n(c2=O)Cc2cc3ccccc3nc2-1
37891    C[C@]12CC[C@H](O)C[C@@]1(O)CC[C@@H]1[C@@H]2CC[...
37892    O=C1c2ccccc2-c2nc3[nH]c(=O)[nH]c(=O)c3c(-c3ccc...
                               ...                        
38365    Cc1cccc(C)c1OCC(=O)NC[C@](O)(Cc1ccccc1)C(=O)N1...
38366    Cc1cccc(C)c1OCC(=O)NC[C@@](O)(Cc1ccccc1)C(=O)N...
38367    Cc1cccc(C)c1OCC(=O)NC[C@@](O)(Cc1ccccc1)C(=O)N...
38368    Cc1cccc(C)c1OCC(=O)NC[C@](O)(Cc1ccccc1)C(=O)N1...
38369    Cc1cccc(C)c1OCC(=O)NC[C@](O)(CCc1ccccc1)C(=O)N...
Name: canonical_smiles, Length: 482, dtype: object


In [16]:
class DataHandler: 
    def find_input_type(self, file_path=DATA_FILE_PATH):
        self.file_path = file_path
        if file_path.split('.')[-1] == 'csv':
            return True
        elif file_path.split('.')[-1] == 'txt':
            return False 
        else: 
            raise ValueError('Unsupported input file. Only .txt and .csv are supported')

In [22]:
data_handler = DataHandler()

data_handler.find_input_type(file_path='test.csv')



True

In [26]:
from tqdm import tqdm
import pickle


# Load all fingerprints with tqdm progress bar
fingerprints, smiles_list, features = [], [], []
for idx in tqdm(range(75), desc="Loading Fingerprints"):
    with open(f'data/fingerprints_chunk_{idx}.pkl', 'rb') as f:
        fps_chunk, smiles_chunk, features_chunk = pickle.load(f)
        fingerprints.extend(fps_chunk)
        smiles_list.extend(smiles_chunk)
        features.extend(features_chunk)

Loading Fingerprints: 100%|██████████| 75/75 [00:00<00:00, 177.31it/s]


['CCNC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3cncnc3)cc2n1',
 'CO[C@@H]1[C@@H](OC(N)=O)[C@@H](O)[C@H](Oc2ccc3c(O)c(NC(=O)c4ccc(O)c(CC=C(C)C)c4)c(=O)oc3c2C)OC1(C)C',
 'CCNC(=O)Nc1cn2c(-c3ncccn3)cc(-c3ccc(C)nc3)cc2n1',
 'CCNC(=O)Nc1cn2c(-c3ncccn3)cc(-c3cccnc3)cc2n1',
 'CCNC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3ccn(CC)c(=O)c3)cc2n1',
 'CCNC(=O)Nc1cn2c(-c3ncccn3)cc(-c3ccc(C)nc3)cc2n1',
 'CCNC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3ccc(OC)nc3)cc2n1',
 'CCNC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3cccnc3)cc2n1',
 'CCNC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3cncnc3)cc2n1',
 'CCNC(=O)Nc1cn2c(-c3ncc(CC)cn3)cc(-c3cccnc3)cc2n1',
 'O=C(NCc1ncc(-c2ccc(Br)cc2)[nH]1)Nc1ncc(Sc2ccccn2)s1',
 'O=C(NCc1ncc(-c2ccc(Cl)c(Cl)c2)[nH]1)Nc1ncc(Sc2ccccn2)s1',
 'Cc1c(Cc2c(Cl)cccc2Cl)c(=O)ccn1CCc1ccc(-c2c[nH]c(CNC(=O)Nc3nc4ccc(S(C)(=O)=O)cc4s3)n2)cc1',
 'O=C(NCc1ncc(-c2cccc(Br)c2)[nH]1)Nc1ncc(Sc2ccccn2)s1',
 'Cc1c(Cc2c(Cl)cccc2Cl)c(=O)ccn1CCc1ccc(-c2c[nH]c(CNC(=O)Nc3ncc(S(C)(=O)=O)s3)n2)cc1',
 'O=C(Nc1ccc(C2CCCCC2)cc1)C1=C(O)CC(Cc2ccccc2)NC1=O',
 'O=C(Nc1ccc